In [ ]:
!pip install langchain-core
!pip install -U langchain-openai
!pip install langgraph

In [ ]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass()

from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4")

from langchain_core.messages import HumanMessage

model.invoke([HumanMessage(content="Hi! I'm Sathapon")])

In [ ]:
model.invoke([HumanMessage(content="What's my name?")])

In [ ]:
from langchain_core.messages import AIMessage

model.invoke(
    [
        HumanMessage(content="Hi! I'm Sathapon"),
        AIMessage(content="Hello Mr. Sathapon! How can I assist you today?"),
        HumanMessage(content="What's my name?"),
    ]
)

In [ ]:
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph

# Define a new graph
workflow = StateGraph(state_schema=MessagesState)


# Define the function that calls the model
def call_model(state: MessagesState):
    response = model.invoke(state["messages"])
    return {"messages": response}


# Define the (single) node in the graph
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

# Add memory
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

# Add threat ID
config = {"configurable": {"thread_id": "abc123"}}

In [ ]:
query = "Hi! I'm Sathapon."

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)

output["messages"][-1].pretty_print()  # output contains all messages in state

In [ ]:
query = "What's my name?"

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

In [ ]:
output["messages"][-1].pretty_print()

In [ ]:
for message in output["messages"]:
    message.pretty_print()

In [ ]:
query = "Do you think you are human?"

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

In [ ]:
for message in output["messages"]:
    message.pretty_print()